In [23]:
import pandas as pd
import utils.misc as misc
from config import *
import yahoo_fin.stock_info as si
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
from glob import glob

In [136]:
def add_ma(df, n):
    col_name = 'DMA'+str(n)
    df[col_name] = df["close"].rolling(n).mean()
    return df
def find_high(df):
    high = max(df['high'])
    high_date = df[df['high'] == high].iloc[0,1]
    # print(high, high_date)
    return (high, high_date)
def find_low(df):
    low = min(df['low'])
    low_date = df[df['low'] == low].iloc[0,1]
    # print(low, low_date)
    return (low, low_date)
def returns(buy_price, sell_price):
    return round((sell_price/buy_price - 1) * 100, 2)

In [138]:
def buy_hold(df, ex_date, next_ex_date):
    output = dict()
    df_buy_hold = df[(df['date'] >= ex_date) & (df['date'] < next_ex_date)]
    buy_price = df_buy_hold.iloc[0].open
    sell_price = df_buy_hold.iloc[-1].close
    output['bh_returns'] = returns(buy_price, sell_price)
    output['bh_high'], x = find_high(df_buy_hold)
    output['bh_dth'] = df_buy_hold.loc[ex_date: x].shape[0]
    output['bh_low'], y = find_low(df_buy_hold)
    output['bh_dtl'] = df_buy_hold.loc[ex_date: y].shape[0]
    output['bh_high_returns'] = returns(buy_price, output['bh_high'])
    return output

In [144]:
def trail(df, SMA):
    sl = df.iloc[0].low
    # print(sl)
    dma_col = 'DMA'+str(SMA)
    df['above_DMA'] = (df['close'] >= df[dma_col])
    for i, row in df.iterrows():
        # print(i, row)
        if row.low < sl:
            # print('SL hit')
            return ('SL', row.date, sl)
        elif row.above_DMA == False:
            return ('DMA'+str(SMA), row.date, row.close)
    return ('In Position', row.date, row.close)

In [145]:
def trail_strategy(df, ex_date, SMA=10):
    output = dict()
    add_ma(df, SMA)
    df_trail = df[df['date'] >= ex_date]
    buy_price = df_trail.iloc[0].open
    exit_method, exit_date, sell_price = trail(df_trail,SMA)
    df_trail = df_trail[df_trail['date'] <= exit_date]
    output['tr_returns'] = returns(buy_price, sell_price)
    output['tr_high'], x = find_high(df_trail)
    output['tr_dth'] = df_trail.loc[exit_date: x].shape[0]
    output['tr_low'], y = find_low(df_trail)
    output['tr_dtl'] = df_trail.loc[exit_date: y].shape[0]
    output['tr_high_returns'] = returns(buy_price, output['tr_high'])
    output['method'] = exit_method
    return output

In [146]:
if __name__ == "__main__":

    ticker = 'META'
    df = pd.read_csv(f"../data/{ticker}_ADJUSTED_OHLC.csv")
    df.drop(columns=['close'], axis=1, inplace=True)
    df.rename(columns={'adjclose': 'close'},inplace=True)
    df.index = df['date']
    
    #input
    ex_date = '2022-01-03'
    next_ex_date = '2022-03-31'

    #holding until next earnings
    temp = buy_hold(df, ex_date, next_ex_date)
    print(temp)

    #trailing using 10 Day Moving Average
    ma_period = 10
    temp = trail_strategy(df, ex_date, ma_period)
    print(temp)

{'bh_returns': -32.65, 'bh_high': 343.0899963378906, 'bh_dth': 2, 'bh_low': 185.82000732421875, 'bh_dtl': 49, 'bh_high_returns': 1.42}
{'tr_returns': -0.33, 'tr_high': 343.0899963378906, 'tr_dth': 1, 'tr_low': 331.8699951171875, 'tr_dtl': 1, 'tr_high_returns': 1.42, 'method': 'SL'}


/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_26215/1101902381.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_DMA'] = (df['close'] >= df[dma_col])


Qullamaggie trail method
1. Buy on ORB
2. Stops at LOD
3. Trail using 10 DMA